In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter

/Users/gordontveito-duncan/Documents/GitHub/prompt-engineering-for-generative-ai-examples/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Custom imports:
from content_collection import collect_serp_data_and_extract_text_from_webpages
from custom_summarize_chain import create_all_summaries, DocumentSummary
from expert_interview_chain import InterviewChain

In [3]:
from dotenv import load_dotenv
# Load environment variables from the .env file
load_dotenv()

True

## Topic Research:

%pip install google-search-results pandas html2text pytest-playwright chromadb nest_asyncio --quiet

!playwright install

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
# Constant variables:
TOPIC = "Neural networks"

In [6]:
# Extract content from webpages into LangChain documents:
text_documents = await collect_serp_data_and_extract_text_from_webpages(topic=TOPIC)

print(text_documents)

In [7]:
# LLM, text splitter + parser:
llm = ChatOpenAI(temperature=0)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1500, chunk_overlap=400
)
parser = PydanticOutputParser(pydantic_object=DocumentSummary)

In [8]:
summaries = await create_all_summaries(text_documents, parser, llm, text_splitter)

Summarizing the data!
Summarizing the data!
Summarizing the data!
Time taken: 3.022839069366455
Finished summarizing the data!
---
Time taken: 3.1113948822021484
Finished summarizing the data!
---
Time taken: 3.238793134689331
Finished summarizing the data!
---


---

## Expert Interview Questions:


In [9]:
interview_chain = InterviewChain(topic=TOPIC, document_summaries=summaries)
interview_questions = interview_chain()

----------------------------------------

## Answer The Interview Questions:

In [10]:
for question in interview_questions.questions:
    print(f"Answer the following question: {question.question}\n", flush=True)
    answer = input(f"Answer the following question: {question.question}\n")
    print('------------------------------------------')
    question.answer = answer

Answer the following question: Can you explain the concept of neural networks and how they mimic the human brain?

------------------------------------------
Answer the following question: What are the key components or layers of a neural network and how do they interact with each other?

------------------------------------------
Answer the following question: How does the training data influence the accuracy and performance improvement of neural networks over time?

------------------------------------------
Answer the following question: In what ways are neural networks considered powerful tools in the fields of computer science and artificial intelligence?

------------------------------------------
Answer the following question: Can you provide insights into the historical evolution of neural networks, especially in terms of periods of popularity and decline?

------------------------------------------


---

## General Article Outline:


In [11]:
from article_outline_generation import BlogOutlineGenerator

blog_outline_generator = BlogOutlineGenerator(topic=TOPIC, questions_and_answers=[item.dict()  
                                                                                  for item in interview_questions.questions ] )
questions_and_answers = blog_outline_generator.questions_and_answers
outline_result = blog_outline_generator.generate_outline(summaries)

Generating the outline...
---
Finished generating the outline!
---


In [19]:
print(outline_result)

title='Understanding Neural Networks: A Deep Dive into Machine Learning' sub_headings=[SubHeading(title='1. Introduction to Neural Networks'), SubHeading(title='2. Components and Layers of Neural Networks'), SubHeading(title='3. The Impact of Training Data on Neural Network Performance'), SubHeading(title='4. The Power of Neural Networks in Computer Science and AI'), SubHeading(title='5. Evolution of Neural Networks: From Past to Present')]


---

## Article Text Generation:


In [12]:
from article_generation import ContentGenerator

content_gen = ContentGenerator(topic=TOPIC, outline=outline_result, questions_and_answers=questions_and_answers)

# Vectorize and store the original webpages:
content_gen.split_and_vectorize_documents(text_documents)

# Create the blog post (this may take several minutes to run, so please be patient):
blog_post = content_gen.generate_blog_post()

Generating the blog post...
---
Finished generating the blog post!
---


In [13]:
blog_post

["# Introduction to Neural Networks\n\nNeural networks, also known as artificial neural networks (ANNs) or simulated neural networks (SNNs), are machine learning models that are inspired by the structure and function of the human brain. They mimic the way biological neurons work together to identify phenomena, weigh options, and arrive at conclusions[^1^].\n\n## Mimicking the Human Brain\n\nNeural networks consist of layers of interconnected nodes, or artificial neurons, which process information in a way similar to biological neurons[^1^]. These nodes are organized into three main layers: the input layer, one or more hidden layers, and the output layer[^1^]. Each node is connected to others and has its own associated weight and threshold[^1^]. If the output of any individual node exceeds the specified threshold value, that node is activated, sending data to the next layer of the network[^1^]. Otherwise, no data is passed along to the next layer[^1^].\n\n## Learning from Data\n\nNeural

-----

## Image Creation:

In [14]:
from image_generation_chain import create_image
image = create_image(outline_result.title)

In [15]:
print(f"The image is ready! The filepath is {image[0]}")

The image is ready! The filepath is 06e2d45.png


In [16]:
pwd

'/Users/gordontveito-duncan/Documents/GitHub/prompt-engineering-for-generative-ai-examples/content/chapter_10'